In [ ]:
# Copyright 2023 The Cirq Developers
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gate with Registers

This package includes a subclass of `cirq.Gate` called `GateWithRegisters`. Instead of operating on a flat list of `cirq.Qid`, this lets the developer define gates in terms of named registers of given widths.

## `Registers`

`Register` objects have a name and a shape. `Registers` is an ordered collection of `Register` with some helpful methods.

In [ ]:
from cirq_ft import Register, Registers

control_reg = Register(name='control', shape=(2,))
target_reg = Register(name='target', shape=(3,))
control_reg, target_reg

In [ ]:
r = Registers([control_reg, target_reg])
r

You can also use the `build` factory method to quickly define a set of registers

In [ ]:
r == Registers.build(
    control=2,
    target=3,
)

### `GateWithRegisters`

In [ ]:
import cirq
from cirq_ft import GateWithRegisters

class MyGate(GateWithRegisters):
    
    @property
    def registers(self):
        return Registers.build(
            control=2,
            target=3,
        )
    
    def decompose_from_registers(self, context, control, target):
        assert len(control) == 2
        assert len(target) == 3
        
        for c in control:
            for t in target:
                yield cirq.CNOT(c, t)
        

In [ ]:
gate = MyGate()
gate

In [ ]:
# Number of qubits is derived from registers
cirq.num_qubits(gate)

The `Registers` object can allocate a dictionary of `cirq.NamedQubit` that we can use to turn our `Gate` into an `Operation`. `GateWithRegisters` exposes an `on_registers` method to compliment Cirq's `on` method where we can use names to make sure each qubit is used appropriately.

In [ ]:
r = gate.registers
quregs = r.get_named_qubits()
quregs

In [ ]:
operation = gate.on_registers(**quregs)
operation

In [ ]:
from cirq.contrib.svg import SVGCircuit
SVGCircuit(cirq.Circuit(operation))

## `GateHelper`

Since `GateWithRegisters` contains enough metadata to derive qubits, an operation, and a circuit we provide a helper class to provide easy access to these quantities.

In [ ]:
import cirq_ft.infra.testing as cq_testing

g = cq_testing.GateHelper(gate)

print('r:', g.r)
print('quregs:', g.quregs)
print('operation:', g.operation)
print('\ncircuit:\n', g.circuit)
print('\n\ndecomposed circuit:\n', cirq.Circuit(cirq.decompose_once(g.operation)))